<a href="https://colab.research.google.com/github/nikhilsai27/NLP/blob/main/NLP_Exp_8_Twitter_Sentiment_Analysis_using_TextBlob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install twint
!pip install transformers

In [22]:
import twint
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [23]:
import os
import asyncio

In [24]:
def analyze_sentiment(text):
    """
    This function analyzes the sentiment of a text using VADER sentiment analyzer.

    Args:
        text: The text string to analyze.

    Returns:
        A dictionary containing sentiment scores (positive, negative, neutral, compound).
    """
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(text)
    return scores

In [25]:
async def main():
    os.environ['TWINT_CONSUMER_KEY'] = "gu4yOjouQeXUDM6580OVkrUqB"
    os.environ['TWINT_CONSUMER_SECRET'] = "fByg0DfubFaiMIMOrSwi5NcgudzJzLo5YGJHwLzQZ7eIdnRONy"
    os.environ['TWINT_ACCESS_TOKEN'] = "1390867607641759746-PVDI7yNjDLL9VFaaOxz83wMuPizF2S"
    os.environ['TWINT_ACCESS_TOKEN_SECRET'] = "jx3HfAtteic20tWCGI5kUwpPIWd9tPmlhukjrgu3ZOlp2"

    c = twint.Config()
    c.Search = "batman"
    c.Limit = 10

    await twint.run.Search(c)

    for tweet in twint.output.tweets:
        tweet_text = tweet.tweet
        sentiment_scores = await analyze_sentiment(tweet_text)

        print(f"Tweet Text: {tweet_text}")
        print(f"Sentiment Scores: {sentiment_scores}")
        print("-"*20)

In [26]:
asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [27]:
!pip install ntscraper

In [28]:
import pandas as pd
from ntscraper import Nitter

/usr/local/lib/python3.10/dist-packages/ntscraper/nitter.py:577: RuntimeWarning: coroutine 'main' was never awaited
  def _get_replied_to(self, tweet):


In [29]:
scraper = Nitter()

def get_tweets(query, mode, size):
  tweets = scraper.get_tweets(query, mode = mode, number = size)
  final_tweets = []

  for tweet in tweets['tweets']:
    data = [tweet['link'], tweet['text'], tweet['date'], tweet['stats']['likes'], tweet['stats']['comments']]
    final_tweets.append(data)

  data = pd.DataFrame(final_tweets, columns = ['link', 'text', 'date', 'Likes', 'Comments'])
  return data

Testing instances: 100%|██████████| 77/77 [01:44<00:00,  1.36s/it]


In [30]:
data = get_tweets("India", "hashtag", 10)
data

INFO:root:No instance specified, using random instance https://nitter.esmailelbob.xyz


,link,text,date,Likes,Comments


In [31]:
import requests
import json

def analyze_sentiment(text):
    url = "https://comprehend-it.p.rapidapi.com/predictions/ml-zero-nli-model"

    payload = {
        "labels": ["positive", "negative", "neutral"],
        "text": text
    }
    headers = {
        "content-type": "application/json",
        "X-RapidAPI-Key": "f92572b58emsh4bc6880188685dbp18f156jsn9743fa271945",
        "X-RapidAPI-Host": "comprehend-it.p.rapidapi.com"
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}"

In [32]:
def analyze_sentiment(text):
    """
    This function analyzes the sentiment of a text using VADER sentiment analyzer.

    Args:
        text: The text string to analyze.

    Returns:
        A dictionary containing sentiment scores (positive, negative, neutral, compound) if successful,
        or None if there's an error.
    """
    analyzer = SentimentIntensityAnalyzer()
    try:
        scores = analyzer.polarity_scores(text)
        return scores
    except Exception as e:
        print(f"Error analyzing sentiment: {e}")
        return None  # Indicate error

sentiments = []
for text in data['text']:
    if isinstance(text, str):
        result = analyze_sentiment(text)
        if result:  # Check if result is not None (error case)
            sentiment_scores = result
            sentiment_str = "; ".join([f"{sent}: {round(score * 100, 2)}%" for sent, score in sentiment_scores.items()])
        else:
            sentiment_str = 'Error: Sentiment analysis failed'  # More informative message
    else:
        sentiment_str = 'Error: Input not text'  # Handle non-string data

    sentiments.append(sentiment_str)


In [33]:
data['Sentiment'] = sentiments
data

,link,text,date,Likes,Comments,Sentiment
